In [76]:
#!pip install pql
import pql
testAll = pql.find("elemets == all(['Si', 'O', 3])")

In [77]:
from optimade.filter import Parser
from lark import Transformer
p = Parser(version=(0, 9, 5))

In [78]:
raw = 'filter=formula_prototype="AB2C3" and nelements=4 and elements="Si, O"'

In [90]:
tree = p.parse(raw)
tree

Tree(start, [Token(KEYWORD, 'filter='), Tree(expression, [Tree(term, [Tree(term, [Tree(term, [Tree(atom, [Tree(comparison, [Token(VALUE, 'formula_prototype'), Token(OPERATOR, '='), Token(VALUE, '"AB2C3"')])])]), Token(AND, 'and'), Tree(atom, [Tree(comparison, [Token(VALUE, 'nelements'), Token(OPERATOR, '='), Token(VALUE, '4')])])]), Token(AND, 'and'), Tree(atom, [Tree(comparison, [Token(VALUE, 'elements'), Token(OPERATOR, '='), Token(VALUE, '"Si, O"')])])])])])

In [112]:
optiMadeToPQLOperatorSwitch = {
    "=":"==",
    "<=":"<=",
    ">=":">=",
    "=>":">=",
    "!=": "!=",
    "<":"<",
    ">":">",
}

def OptiMadeToPQLOperatorValidator(x):
    return optiMadeToPQLOperatorSwitch[x]


class DecodeTransformer(Transformer):
    def comparison(self, args):
        A = args[0]
        B = args[2]
        operator = OptiMadeToPQLOperatorValidator(args[1])
        return A + operator + B
    def atom(self, args):
        return args[0]
    def term(self, args):
        result = ""
        for arg in args:
            result = result + " " + arg
        return result.lstrip()
    def expression(self, args):
        return args[0]
    def start(self, args):
        return args[1]

In [113]:
cleandedPQL = DecodeTransformer().transform(tree)
cleandedPQL

'formula_prototype=="AB2C3" and nelements==4 and elements=="Si, O"'

In [114]:
def removeFilter(raw):
    if(raw.find("filter =") != -1):
        return raw[len("filter ="):]
    elif(raw.find("filter=") != -1):
        return raw[len("filter="):]
    else:
        return raw
    
def insert(original, index, insertion):
    result = original[:index] + insertion + original[index:]
    return result

def onlyEqual(string, index):
    return string[index] == "=" and not (string[index-1] == "<" 
                     or string[index+1] == ">" 
                     or string[index-1] == "=" 
                     or string[index+1] == "=") 


def combineMultiple(string, index):
    firstQuoteIndex = 0
    for firstQuoteIndex in reversed(range(0, index)):
        if(ord(string[firstQuoteIndex]) == 34):
            break
    lastQuoteIndex = index
    for lastQuoteIndex in range(index, len(string)):
        if(ord(string[lastQuoteIndex])== 34):
            break
    insertion = string[firstQuoteIndex+1:lastQuoteIndex]
    insertion = insertion.split(",")
    return string[:firstQuoteIndex] + 'all({})'.format(insertion) + string[lastQuoteIndex+1:]

In [123]:
"""
Parsing the input(Optimade form) to Materials Project's MongoDB Query

ex 1: (change "=" to ==,  "" to '', and "Si, O" to all(['Si', 'O']))
input -> filter=formula_prototype="AB2C3"  AND  nelements=4  AND  elements="Si,O"
output ->  formula_prototype='AB2C3' and nelements==4 and elements==all(['Si', 'O'])

ex2:(no change)
input -> filter=(x<3  and  y!=2)  or  not  z<0
output -> (x<3  and  y!=2)  or  not  z<0

"""
def parseInput(PQL):
#     filterRemoved = removeFilter(raw)
#     lowerCased = filterRemoved.replace("AND","and").replace("OR", "or")
#     result = lowerCased
    length = len(PQL)
    i = 0
    while(i < length):
#         if(onlyEqual(result, i)):
#             result = insert(result, i, "=")
#             length = len(result)
        if(PQL[i] == ","):
            PQL = combineMultiple(PQL, i)
        i = i + 1 
    return PQL

In [124]:
# x = parseInput('filter=formula_prototype="AB2C3" and nelements=4 AND elements="Si,O"')
x = parseInput(cleandedPQL)
print(x)
pql.find(x)

formula_prototype=="AB2C3" and nelements==4 and elements==all(['Si', ' O'])


{'$and': [{'formula_prototype': 'AB2C3'},
  {'nelements': 4},
  {'elements': {'$all': ['Si', ' O']}}]}

In [125]:
pql.find("formula_prototype=='AB2C3' and nelements==4 and elements==all(['Si', 'O'])")
pql.find("x<=3 ")
pql.find("(x<=3  and  y!=2)  or  not z<0")

{'$or': [{'$and': [{'x': {'$lte': 3}}, {'y': {'$ne': 2}}]},
  {'z': {'$not': {'$lt': 0}}}]}